In [1]:
/*
     Step #1 :  Import Libraries
 */
import org.apache.spark._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql._
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._

Intitializing Scala interpreter ...

Spark Web UI available at http://cluster2-m:8088/proxy/application_1621161459144_0001
SparkContext available as 'sc' (version = 3.1.1, master = yarn, app id = application_1621161459144_0001)
SparkSession available as 'spark'


import org.apache.spark._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql._
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._


In [2]:
val vAR_spark = SparkSession.builder
  .config("spark.jars.packages", "com.google.cloud.spark:spark-bigquery-with-dependencies_2.11:0.19.1")
  .getOrCreate()

vAR_spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1e3d0318


In [3]:
val vAR_bucket = "dataproc-staging-us-central1-363181475522-it3j8dl5"
vAR_spark.conf.set("temporaryGcsBucket", vAR_bucket)

vAR_bucket: String = dataproc-staging-us-central1-363181475522-it3j8dl5


In [4]:
val data = Seq(("divine-course-311407.Errors.hadoop_hdfs_namenode_20210509"),
                ("divine-course-311407.Errors.hadoop_hdfs_datanode_20210509"),
              ("divine-course-311407.Errors.hadoop_hdfs_secondarynamenode_20210509"),
              ("divine-course-311407.Errors.hadoop_mapred_historyserver_20210509"),
              ("divine-course-311407.Errors.hadoop_yarn_nodemanager_20210509"),
              ("divine-course-311407.Errors.hadoop_yarn_resourcemanager_20210509"),
              ("divine-course-311407.Errors.hadoop_yarn_timelineserver_20210509"),
              ("divine-course-311407.Errors.yarn_userlogs_20210509"))

data: Seq[String] = List(divine-course-311407.Errors.hadoop_hdfs_namenode_20210509, divine-course-311407.Errors.hadoop_hdfs_datanode_20210509, divine-course-311407.Errors.hadoop_hdfs_secondarynamenode_20210509, divine-course-311407.Errors.hadoop_mapred_historyserver_20210509, divine-course-311407.Errors.hadoop_yarn_nodemanager_20210509, divine-course-311407.Errors.hadoop_yarn_resourcemanager_20210509, divine-course-311407.Errors.hadoop_yarn_timelineserver_20210509, divine-course-311407.Errors.yarn_userlogs_20210509)


In [5]:
for( a <- 0 to 7 ){ 
    val LogErrs = vAR_spark.read.format("bigquery")
    .load(data(a))
    .select("jsonPayload.message","timestamp")
    .cache()
    
    if (a==0){LogErrs.write 
  .format("bigquery") 
  .option("table", "divine-course-311407:Errors.ErrorMsg") 
  .option("temporaryGcsBucket",vAR_bucket)
  .mode("overwrite") 
  .save()}
    
    else{LogErrs.write 
  .format("bigquery") 
  .option("table", "divine-course-311407:Errors.ErrorMsg") 
  .option("temporaryGcsBucket",vAR_bucket)
  .mode("append") 
  .save()}
    
      }  

In [6]:
val Messages = vAR_spark.read.format("bigquery")
    .load("divine-course-311407:Errors.ErrorMsg")
    .cache()

Messages: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [message: string, timestamp: timestamp]


In [7]:
Messages.show(false)

+---------------------------+-----------------------+
|message                    |timestamp              |
+---------------------------+-----------------------+
|RECEIVED SIGNAL 1: SIGHUP  |2021-05-09 10:24:14.943|
|RECEIVED SIGNAL 1: SIGHUP  |2021-05-09 05:32:33.531|
|RECEIVED SIGNAL 1: SIGHUP  |2021-05-09 13:19:12.254|
|RECEIVED SIGNAL 15: SIGTERM|2021-05-09 10:24:14.998|
|RECEIVED SIGNAL 15: SIGTERM|2021-05-09 05:32:33.494|
|RECEIVED SIGNAL 15: SIGTERM|2021-05-09 13:19:12.212|
|RECEIVED SIGNAL 1: SIGHUP  |2021-05-09 05:34:19.536|
|RECEIVED SIGNAL 1: SIGHUP  |2021-05-09 10:26:01.848|
|RECEIVED SIGNAL 1: SIGHUP  |2021-05-09 05:34:19.484|
|RECEIVED SIGNAL 1: SIGHUP  |2021-05-09 10:26:01.861|
|RECEIVED SIGNAL 15: SIGTERM|2021-05-09 05:34:19.5  |
|RECEIVED SIGNAL 15: SIGTERM|2021-05-09 10:26:01.811|
|RECEIVED SIGNAL 15: SIGTERM|2021-05-09 05:34:19.488|
|RECEIVED SIGNAL 15: SIGTERM|2021-05-09 10:26:01.856|
|RECEIVED SIGNAL 1: SIGHUP  |2021-05-09 05:32:33.003|
|RECEIVED SIGNAL 1: SIGHUP  

In [8]:
var splitMsg = Messages.withColumn("Description",split(col("message"),":").getItem(0))
    .withColumn("ErrorName",split(col("message"),":").getItem(1))
    .withColumn("ErrorName",ltrim(col("ErrorName")))
    .withColumn("Date",split(col("timestamp")," ").getItem(0))
    .withColumn("Time",split(col("timestamp")," ").getItem(1))
    .drop("message","timestamp")

splitMsg: org.apache.spark.sql.DataFrame = [Description: string, ErrorName: string ... 2 more fields]


In [9]:
splitMsg.show(4)

+------------------+---------+----------+------------+
|       Description|ErrorName|      Date|        Time|
+------------------+---------+----------+------------+
| RECEIVED SIGNAL 1|   SIGHUP|2021-05-09|10:24:14.943|
| RECEIVED SIGNAL 1|   SIGHUP|2021-05-09|05:32:33.531|
| RECEIVED SIGNAL 1|   SIGHUP|2021-05-09|13:19:12.254|
|RECEIVED SIGNAL 15|  SIGTERM|2021-05-09|10:24:14.998|
+------------------+---------+----------+------------+
only showing top 4 rows



In [10]:
def frequencies(df: DataFrame) = df.groupBy("ErrorName","Description").count().orderBy('count.desc)
var vAR_ErrorCount = frequencies(splitMsg)

frequencies: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]
vAR_ErrorCount: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ErrorName: string, Description: string ... 1 more field]


In [11]:
vAR_ErrorCount = vAR_ErrorCount.withColumnRenamed("ErrorName","ErrNam")

vAR_ErrorCount: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ErrNam: string, Description: string ... 1 more field]


In [35]:
val transform_expr = "transform(ts_array, (x,i) -> (bigint((to_timestamp(ts_array[i+1]))) - (bigint(to_timestamp(x))))/60)"
def Frq(df : DataFrame) = df.groupBy("ErrorName").agg(array_sort(collect_list(col("Time"))).alias("ts_array")) 
    .withColumn("frequency", expr(transform_expr))
    .drop("ts_array")
val frq = Frq(splitMsg)

transform_expr: String = transform(ts_array, (x,i) -> (bigint((to_timestamp(ts_array[i+1]))) - (bigint(to_timestamp(x))))/60)
Frq: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
frq: org.apache.spark.sql.DataFrame = [ErrorName: string, frequency: array<double>]


In [31]:
val vAR_dm = frq.join(vAR_ErrorCount).where(frq("ErrorName") === vAR_ErrorCount("ErrNam"))

vAR_dm: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ErrorName: string, frequency: array<interval> ... 3 more fields]


In [32]:
vAR_dm.show(false)

+------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------+-----------------------------------------------------------+-----+
|ErrorName                     |frequency                                                                                                                                                                                                                                                                                                                                                        

In [88]:
vAR_ErrorCount.show(false)

+-------------------------------+-----------------------------------------------------------+-----+
|ErrNam                         |Description                                                |count|
+-------------------------------+-----------------------------------------------------------+-----+
| SIGTERM                       |RECEIVED SIGNAL 15                                         |17   |
| SIGHUP                        |RECEIVED SIGNAL 1                                          |15   |
|null                           |RECEIVED SIGNAL TERM                                       |8    |
| sleep interrupted             |ExpiredTokenRemover received java.lang.InterruptedException|5    |
| java.lang.InterruptedException|Returning, interrupted                                     |3    |
+-------------------------------+-----------------------------------------------------------+-----+



In [33]:
vAR_dm.write 
  .format("bigquery") 
  .option("table", "divine-course-311407:Errors.ErrFrq1") 
  .option("temporaryGcsBucket",vAR_bucket)
  .mode("overwrite") 
  .save()

java.lang.RuntimeException:  Failed to write to BigQuery